In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Atherosclerosis"
cohort = "GSE123086"

# Input paths
in_trait_dir = "../../input/GEO/Atherosclerosis"
in_cohort_dir = "../../input/GEO/Atherosclerosis/GSE123086"

# Output paths
out_data_file = "../../output/preprocess/Atherosclerosis/GSE123086.csv"
out_gene_data_file = "../../output/preprocess/Atherosclerosis/gene_data/GSE123086.csv"
out_clinical_data_file = "../../output/preprocess/Atherosclerosis/clinical_data/GSE123086.csv"
json_path = "../../output/preprocess/Atherosclerosis/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Gene Expression Data Availability
# Looking at the background info: mentions microarrays and RNA extraction, suggesting gene expression data is available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# Trait: For atherosclerosis, primary diagnosis is in row 1
trait_row = 1

# Age: Available in rows 3 and 4
age_row = 3

# Gender: Available in row 2 (and also some values appear in row 3)
gender_row = 2

# 2.2 Data Type Conversion

def convert_trait(value):
    """Convert trait value to binary format."""
    if value is None or pd.isna(value):
        return None
    
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # For Atherosclerosis trait
    if "ATHEROSCLEROSIS" in value:
        return 1
    else:
        return 0

def convert_age(value):
    """Convert age value to continuous format."""
    if value is None or pd.isna(value):
        return None
    
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender value to binary format (0=female, 1=male)."""
    if value is None or pd.isna(value):
        return None
    
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if value.upper() == "MALE":
        return 1
    elif value.upper() == "FEMALE":
        return 0
    else:
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Conduct initial filtering and save metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Create output directories if they don't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Use the function from the library to extract clinical features
    clinical_features_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview = preview_df(clinical_features_df)
    print("Clinical features preview:", preview)
    
    # Save the clinical data
    clinical_features_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [ ]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


### Step 4: Gene Identifier Review

In [ ]:
# The identifiers shown are numeric values ('1', '2', '3', etc.)
# These are not standard human gene symbols, which would typically be alphanumeric
# (like "BRCA1", "TP53", "APOE", etc.)
# These appear to be probe or feature IDs that need to be mapped to actual gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Check the first few rows of the SOFT file to better understand its structure
print("\nChecking the SOFT file structure for gene symbols:")
gene_symbol_data = []
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if i < 1000:  # Examine more lines to find gene symbol information
            if "GENE_SYMBOL" in line or "gene_symbol" in line.lower() or "symbol" in line.lower():
                print(line.strip())
                gene_symbol_data.append(line.strip())
        else:
            break

print("\nSearching for gene symbols in the SOFT file...")
# Look for table headers that could contain gene symbol information
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if i < 1000 and "!platform_table_begin" in line:
            # Get the next line which should contain column headers
            header_line = next(f, "").strip()
            print(f"Found platform table headers: {header_line}")
            break

# We need to create a more appropriate mapping
# First, check if we can extract gene symbols from the Entrez Gene IDs
# Use the extract_human_gene_symbols function from the library
print("\nAttempting to map Entrez Gene IDs to gene symbols...")

# Create a basic mapping dataframe with probe IDs and Entrez Gene IDs
mapping_data = gene_annotation[['ID', 'ENTREZ_GENE_ID']].copy()
mapping_data = mapping_data.dropna(subset=['ENTREZ_GENE_ID'])
mapping_data = mapping_data.rename(columns={'ENTREZ_GENE_ID': 'Gene'})

# Filter out any empty gene values
mapping_data = mapping_data[mapping_data['Gene'] != '']

# Validate the mapping data
print(f"Mapping data shape: {mapping_data.shape}")
print("Sample mapping data:")
print(preview_df(mapping_data, n=5))

# Check if there's additional mapping information in the annotation
print("\nLooking for gene symbols in annotation data...")
for col in gene_annotation.columns:
    sample_values = gene_annotation[col].head(10).tolist()
    print(f"Column '{col}' sample values: {sample_values}")

# Count how many unique probes we have for mapping
unique_probes = mapping_data['ID'].nunique()
print(f"\nNumber of unique probes for mapping: {unique_probes}")

# Count how many probes map to at least one gene identifier
genes_mapped = mapping_data.shape[0]
total_probes = gene_annotation.shape[0]
mapping_percentage = (genes_mapped / total_probes) * 100 if total_probes > 0 else 0
print(f"Found gene identifiers for {genes_mapped} out of {total_probes} probes ({mapping_percentage:.2f}%)")

# The mapping contains Entrez Gene IDs which are numeric identifiers from NCBI
# We'll use these directly since it appears no human-readable gene symbols are available in the annotation
gene_mapping = mapping_data

# Display the final mapping we'll use
print("\nFinal gene mapping sample:")
print(preview_df(gene_mapping, n=5))


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Analyze the gene identifiers and annotation to decide on mapping strategy
# From previous steps, we see the gene identifiers in the expression data are numeric IDs ('1', '2', '3', etc.)
# The gene annotation has 'ID', 'ENTREZ_GENE_ID', and 'SPOT_ID' columns
# The 'ID' in gene annotation corresponds to the probe IDs in the expression data
# The 'ENTREZ_GENE_ID' contains Entrez Gene IDs which we'll use as gene identifiers

# 2. Create a gene mapping dataframe
gene_mapping = gene_annotation[['ID', 'ENTREZ_GENE_ID']].copy()
gene_mapping = gene_mapping.dropna(subset=['ENTREZ_GENE_ID'])
gene_mapping = gene_mapping.rename(columns={'ENTREZ_GENE_ID': 'Gene'})

# Display the gene mapping
print(f"Gene mapping shape: {gene_mapping.shape}")
print("Sample of gene mapping dataframe:")
print(preview_df(gene_mapping))

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
# We need to handle the issue with apply_gene_mapping which expects gene symbols

# First, select only the rows in gene_mapping that correspond to probes in our gene_data
valid_mapping = gene_mapping[gene_mapping['ID'].isin(gene_data.index)]
print(f"Number of probes in gene_data that have mapping: {len(valid_mapping)}")

# Create a simpler mapping function that preserves the Entrez Gene IDs
def map_probes_to_genes(expression_df, mapping_df):
    """Maps probe-level expression to gene-level expression using Entrez Gene IDs."""
    # Ensure mapping only includes probes that exist in expression data
    mapping_df = mapping_df[mapping_df['ID'].isin(expression_df.index)].copy()
    
    # Set the probe ID as index for joining
    mapping_df.set_index('ID', inplace=True)
    
    # Get all sample columns (all columns in expression_df)
    sample_cols = expression_df.columns.tolist()
    
    # Create a mapping dictionary from probe to gene
    probe_to_gene = mapping_df['Gene'].to_dict()
    
    # Initialize a dictionary to collect gene expression values
    gene_expression = {}
    gene_counts = {}
    
    # Process each probe's expression
    for probe_id, row in expression_df.iterrows():
        if probe_id in probe_to_gene:
            gene = probe_to_gene[probe_id]
            
            # Initialize gene entry if not present
            if gene not in gene_expression:
                gene_expression[gene] = {col: 0 for col in sample_cols}
                gene_counts[gene] = 0
            
            # Add this probe's expression to the gene
            for col in sample_cols:
                gene_expression[gene][col] += row[col]
            
            gene_counts[gene] += 1
    
    # Create a dataframe from the collected expression values
    gene_df = pd.DataFrame.from_dict(gene_expression, orient='index')
    
    # Average the expression by the number of probes per gene
    for gene, count in gene_counts.items():
        gene_df.loc[gene] = gene_df.loc[gene] / count
    
    return gene_df

# Apply the mapping function
gene_data = map_probes_to_genes(gene_data, gene_mapping)

# Display the resulting gene expression data
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First 5 gene IDs in the mapped data:")
print(gene_data.index[:5])
print("Sample of gene expression data (first 5 genes, first 5 columns):")
print(gene_data.iloc[:5, :5])

# Create output directory if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)

# Save the gene expression data
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


### Step 7: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols using NCBI database
print("Normalizing gene symbols...")
gene_data = pd.read_csv(out_gene_data_file, index_col=0)
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {gene_data.shape}")
print("First 10 normalized gene symbols:")
print(gene_data.index[:10])

# Save the normalized gene data
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to: {out_gene_data_file}")

# 2. Extract and prepare clinical data from the matrix file
print("\nPreparing clinical data...")

# Get the clinical data rows
_, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
_, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# Process clinical data using the parameters defined in Step 2
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=0,  # From Step 2: trait_row = 0
    convert_trait=convert_trait,  # Function defined in Step 2
    age_row=None,  # From Step 2: age_row = None
    convert_age=None,
    gender_row=None,  # From Step 2: gender_row = None
    convert_gender=None
)

print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to: {out_clinical_data_file}")

# 3. Link clinical and genetic data
print("\nLinking clinical and genetic data...")
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
if linked_data.shape[0] > 0 and linked_data.shape[1] > 5:
    print(linked_data.iloc[:5, :5])
else:
    print(linked_data)

# 4. Handle missing values
print("\nHandling missing values...")
linked_data_clean = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data_clean.shape}")

# 5. Check for bias in the dataset
print("\nChecking for bias in dataset features...")
is_biased, linked_data_clean = judge_and_remove_biased_features(linked_data_clean, trait)

# 6. Conduct final quality validation
note = "This GSE57691 dataset contains gene expression data from patients with abdominal aortic aneurysm (AAA) and aortic occlusive disease (AOD) compared to control subjects. The dataset focuses on atherosclerosis-related vascular changes."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data_clean,
    note=note
)

# 7. Save the linked data if it's usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data_clean.to_csv(out_data_file, index=True)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for associative studies. Linked data not saved.")